# Script to fit the model to experiments of Aggregation/Breakup of cyanobacterial colonies
### Experiments: Size distribution of Microcystis V163 colonies in turbulent cone-and-plate shear 
### Model: Classes method for size distribution in homogeneous turbulence
### Yuri Sinzato - PhD Student - University of Amsterdam
### Last version: 26-06-2023

#### Brief description of the routine:
- This routine is used to fit the numerical model of aggregation/breakup of aggregates to the experimental results of cyanobacterial colonies in shear
- The experimental results display the time-evolution of the size distribution (and its moments) of Microcystis V163 colonies in tubulent cone-and-plate shear for various concentrations and turbulence intensities
- The numerical model simulates the evolution of the size distribution of two populations (division formed colonies and aggregated colonies) subjected to aggregation and breakage 
- The parameters of the numerical model are fitted to the experiments with a least square minimization of the main descriptors of the size distribution (medians and main percentiles). 
- The numerical model is run with the executable 'Numerical_Model/agg_bre.exe.'
- The initial size distribution is extracted from the experimental data, and supplied to the model via the text file 'Numerical_Model/Temporary/csd_initial.txt'
- The parameters are supplied to the model via the text file 'Numerical_Model/Temporary/parameters.txt'
- The results of the simulation are returned in the file via the text file 'Numerical_Model/Temporary/r_data.txt'
- The numerical results must be interpolated in the time to allow direct comparison with the experimental data
- The minimization is made with the library scipy.optimize

In [22]:
### Libraries
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pims
import os
import scipy
from IPython.display import clear_output
#import time



## Functions

In [23]:

### Function to smoothen a noisy initial size distribution and return the interpolated value 
def smooth_cmd(cmd_exp,r_bin_num,r_bin_exp):
    cmd_exp_smooth = np.zeros(N_r_exp)
    cmd_num = np.zeros(N_r)
    for i in range(N_r_exp):     
        #for j in range(max(math.floor(i*h_r/5.0),1)): # Smooth
        for j in range(1):  # Not smooth
            cmd_exp_smooth[i] = sum(cmd_exp[i-j:min(i+j+1,N_r_exp)])/(1+2*j)
    cmd_num = np.interp(r_bin_num,r_bin_exp,cmd_exp_smooth)
    return cmd_num

In [24]:
### Function to import experimental data file, run simulation and calculate penalty function
def penalty_function(folder_path,phi_o,epsilon,fit_par):
    #  fit_par is an array with the following order parameter = [alpha,ep_s1,ep_s2,q]
    ### Import initial experimental csd data file
    exp_file_path = folder_path+'csd_data.txt'
    r_bin_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=2,usecols=(0),max_rows = N_r_exp) # Read experimental r_bin centers
    cmd_s_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=2,usecols=(2),max_rows = N_r_exp) # Read experimental cmd for small colonies
    cmd_l_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=2,usecols=(3),max_rows = N_r_exp)  # Read experimental cmd for large colonies
    
    ### Import time evolution of statistical descriptors of experimental data
    exp_file_path = folder_path+'r_data.txt'
    time_array_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(0),max_rows = N_t_fit)  # Create array for time points
    frac_small_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(7),max_rows = N_t_fit)  # Create array for fraction of small colonies (kind1)
    r_med1_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(8),max_rows = N_t_fit)  # Create array for median of small colonies (kind1)
    lr_med1_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(9),max_rows = N_t_fit)  # Create array for percentile 25 of small colonies (kind1)
    hr_med1_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(10),max_rows = N_t_fit)  # Create array for percentile 75 of small colonies(kind1)
    r_med2_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(11),max_rows = N_t_fit)  # Create array for median of large colonies (kind2)
    lr_med2_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(12),max_rows = N_t_fit)  # Create array for percentile 25 of large colonies (kind2)
    hr_med2_exp = np.genfromtxt(exp_file_path,dtype='float',skip_header=1,usecols=(13),max_rows = N_t_fit)  # Create array for percentile 75 of large colonies (kind2)


    alpha = fit_par[0] # Stickiness
    ep_s1 = fit_par[1] # Critical dissipation energy small colonies
    ep_s2 = fit_par[2] # Critical dissipation energy large colonies
    q1 = fit_par[3] # Exponent of critical size for small colonies
    q2 = fit_par[4] # Exponent of critical size for large colonies
    #alpha = alpha_i # Stickiness
    #ep_s1 = ep_s1_i # Critical dissipation energy small colonies
    #ep_s2 = ep_s2_i #2.02e4 # Critical dissipation energy large colonies
    #q = q_i # Exponent of critical size
    
    ### Write parameters file
    para_file = open('Numerical_Model/Temporary/parameters.txt', 'w')  # Size distributions 
    para_file.write('N_a \t {:d} \n'.format(N_a) )
    para_file.write('N_r \t {:d} \n'.format(N_r) )
    para_file.write('h_r \t {:12.6f} \n'.format(h_r) )
    para_file.write('t_exp \t {:12.6f} \n'.format(t_exp) )
    para_file.write('N_tmax \t {:d} \n'.format(N_tmax) )
    para_file.write('Dt_i \t {:12.6f} \n'.format(Dt_i) )
    para_file.write('Dt_l \t {:12.6f} \n'.format(Dt_l) )
    para_file.write('Dt_u \t {:12.6f} \n'.format(Dt_u) )
    para_file.write('tol_csd \t {:.5e} \n'.format(tol_csd) )
    para_file.write('mu \t {:12.6f} \n'.format(mu) )
    para_file.write('alpha \t {:.5e} \n'.format(alpha) )
    para_file.write('phi_o \t {:.5e} \n'.format(phi_o) )
    para_file.write('ep_s1 \t {:.5e} \n'.format(ep_s1) )
    para_file.write('ep_s2 \t {:.5e} \n'.format(ep_s2) )
    para_file.write('d_f \t {:12.6f} \n'.format(d_f) )
    para_file.write('q \t {:12.6f} \n'.format(q1) )
    para_file.write('q \t {:12.6f} \n'.format(q2) )
    para_file.write('epsilon \t {:.5e} \n'.format(epsilon) )
    para_file.write('nu \t {:.5e} \n'.format(nu) )
    para_file.close()

    ### Write initial csd data file for simulation
    #Generate initial num csd by smoothing exp csd
    r_bin_num = np.linspace(0.5*h_r,(N_r+0.5)*h_r,N_r,endpoint = False)
    cmd_s_num = smooth_cmd(cmd_s_exp,r_bin_num,r_bin_exp) # Smoothing function in csd of small colonies
    if sum(cmd_s_num) > 0:
        cmd_s_num = cmd_s_num*sum(cmd_s_exp)/sum(cmd_s_num) # Correct normalization
    cmd_l_num = smooth_cmd(cmd_l_exp,r_bin_num,r_bin_exp) # Smoothing function in csd of large colonies
    if sum(cmd_l_num) > 0:
        cmd_l_num = cmd_l_num*sum(cmd_l_exp)/sum(cmd_l_num) # Correct normalization
    # Write in text file
    csd_initial_file = open('Numerical_Model/Temporary/csd_initial.txt', 'w')  # Size distributions 
    csd_initial_file.write('#Initial Normalized size distribution\n')
    csd_initial_file.write('#(1)Block numbe            (2)t(s)                (3)r      (4)csd_f(norm)     (5)csd_m(kind1-norm (6)csd_m(kind2-norm\n')
    for i in range(N_r):
        csd_initial_file.write('{:15d}'.format(0))
        csd_initial_file.write('{:20.6f}'.format(0))
        csd_initial_file.write('{:20.10f}'.format(r_bin_num[i]))
        csd_initial_file.write('{:20.10f}'.format(cmd_s_num[i]))
        csd_initial_file.write('{:20.10f}'.format(cmd_s_num[i]))
        csd_initial_file.write('{:20.10f}\n'.format(cmd_l_num[i]))
    csd_initial_file.close()

    ### Run simulation
    os.system("wsl ./Numerical_Model/agg_bre.exe")

    ### Import time evolution of statistical descriptors of numerical data - Data is interpolated to fit format of experimental data
    file_name = 'Numerical_Model/Temporary/r_data.txt'  # File path
    time_array_num = np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(0))  # Create array for time points
    frac_small_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(7)))  # Create array for fraction of small colonies (kind1)
    r_med1_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(8)))  # Create array for median of small colonies (kind1)
    lr_med1_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(9)))  # Create array for percentile 25 of small colonies (kind1)
    hr_med1_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(10)))  # Create array for percentile 75 of small colonies(kind1)
    r_med2_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(11)))  # Create array for median of large colonies (kind2)
    lr_med2_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(12)))  # Create array for percentile 25 of large colonies (kind2)
    hr_med2_num = np.interp(time_array_exp,time_array_num,np.genfromtxt(file_name,dtype='float',skip_header=1,usecols=(13)))  # Create array for percentile 75 of large colonies (kind2)

    ### Calculate penalty function - squared difference btw num and exp statistical descriptors
    penalty = 0
    penalty += np.sum(np.abs(frac_small_num - frac_small_exp))
    penalty += np.sum(frac_small_exp*np.abs(r_med1_num - r_med1_exp)/np.abs(r_med1_num))
    penalty += np.sum(frac_small_exp*np.abs(lr_med1_num - lr_med1_exp)/np.abs(r_med1_num))
    penalty += np.sum(frac_small_exp*np.abs(hr_med1_num - hr_med1_exp)/np.abs(r_med1_num))
    penalty += np.sum((np.ones(len(frac_small_exp))-frac_small_exp)*np.abs(r_med2_num - r_med2_exp)/np.abs(r_med2_num))
    penalty += np.sum((np.ones(len(frac_small_exp))-frac_small_exp)*np.abs(lr_med2_num - lr_med2_exp)/np.abs(r_med2_num))
    penalty += np.sum((np.ones(len(frac_small_exp))-frac_small_exp)*np.abs(hr_med2_num - hr_med2_exp)/np.abs(r_med2_num))

    print('Folder: '+str(folder_path))
    print('penalty_function:'+str(penalty))
    
    return penalty

In [25]:
### Function to iterate penalty function through various data files

def total_penalty(fit_par): # Selected data file
    total_penalty_value = 0
    for i in range(3,4):
        total_penalty_value += penalty_function(folder_paths[i],phi_o_values[i],epsilon_values[i],fit_par)
    print('Total penalty function: '+str(total_penalty_value))
    print('Fitting parameters: '+str(fit_par))
    return total_penalty_value

def total_penalty_bre(fit_par): # Only breakage data files
    total_penalty_value = 0
    fit_par_comb = np.concatenate((fit_par_i[0:2],fit_par[0:1],fit_par_i[3:4],fit_par[1:2]))
    for i in [0,1,3,4]:
        total_penalty_value += penalty_function(folder_paths[i],phi_o_values[i],epsilon_values[i],fit_par_comb)
    
    print('Total penalty function: '+str(total_penalty_value))
    print('Fitting parameters: '+str(fit_par_comb))
    return total_penalty_value

def total_penalty_agg(fit_par): # Only aggregation data files
    total_penalty_value = 0
    fit_par_comb = np.concatenate((fit_par[0:2],fit_par_i[2:3],fit_par[2:3],fit_par_i[4:5]))
    for i in range(5,9):
        total_penalty_value += penalty_function(folder_paths[i],phi_o_values[i],epsilon_values[i],fit_par_comb)
    print('Total penalty function: '+str(total_penalty_value))
    print('Fitting parameters: '+str(fit_par_comb))
    return total_penalty_value

## Parameters

In [26]:
# Physical parameters
mu = 0.0e-4 # Growth rate
d_f = 2.09 # 3-D fractal dimension
nu = 1.002e-6 # Kinematic viscosity

# Binning parameters - Exp
N_t_fit = 25 ## Time points to use in fit
h_r_exp = 0.5 ## Bin size for experimental data
N_r_exp = 400 ## Number of bins for experimental data

# Binning parameters - Num
R_a = 10.0 # Range aggregative bins
R_r = 80.0 # Range of total bins - Should be at least twice the aggregative range
h_r = 0.5 # Bin size for numerical model
N_a = (np.ceil(R_a/h_r)).astype(int) # Number of aggregative bins
N_r = (np.ceil(R_r/h_r)).astype(int)  # Total number of bins

# Numerical routine parameters 
t_exp = 20000 # Simulation time
N_tmax = 10000000 # Maximum number of time steps in simulation
Dt_i = 1000 # Initial time step (non-dimensional)
Dt_l = 1 # Minimum time step (non-dimensional)
Dt_u = 1e5 # Maximum time step (non-dimensional)
tol_csd = 5e-4 # Tolerance for mass distribution




### Experimental data files

In [27]:
folder_paths = np.array(['Bre_100s_c1e6_23-11-22/Data/',
                         'Bre_500s_c1e6_15-3-2/Data/',
                         'Bre_800s_c1e6_11-4-23/Data/',
                         'Bre_1000s_c1e6_11-1-23/Data/',
                         'Bre_1200s_c1e6_1-5-23/Data/',
                         'Agg_100s_c1e6_24-11-22/Data/',
                         'Agg_100s_c2e6_2-12-22/Data/',
                         'Agg_100s_c5e6_6-1-23/Data/',
                         'Agg_1000s_c1e6_12-1-23/Data/',
                         'Bre_100s_c2e6_8-12-22/Data/',
                         'Bre_100s_c5e6_1-12-22/Data/'
                         ])  # File path
d_eq1 = 5.5 # Single cell equivalent circular diameter in um (Calculated from projected area)
v1 = math.pi*d_eq1**3/6 # Single cell equivalent volume in um3 (Calculated from equivalent circular diameter)
phi_o_values = np.array([1.0e6*v1*1e-12,
                         1.0e6*v1*1e-12,
                         1.0e6*v1*1e-12,
                         1.0e6*v1*1e-12,
                         1.0e6*v1*1e-12,
                         1.0e6*v1*1e-12,
                         2.0e6*v1*1e-12,
                         5.0e6*v1*1e-12,
                         1.0e6*v1*1e-12,
                         2.0e6*v1*1e-12,
                         5.0e6*v1*1e-12]) # Volume fraction of cells
epsilon_values = np.array([0.0192,
                          1.079,
                          3.493,
                          5.784,
                          9.627,
                          0.0192,
                          0.0192,
                          0.0192,
                          5.784,
                          0.0192,
                          0.0192]) # Energy dissipation rate (m2/s3)


## Minimization routine

In [28]:
#Initial guess of fitting parameters
alpha_i =  2.265e-02 #2.44200000e-02   # Stickiness
ep_s1_i =  3.412e-02 # 4.28100000e-02 # Critical dissipation energy small colonies
ep_s2_i =  3.142e+01 # 5.43597852e+01  # Critical dissipation energy large colonies
q1_i = 4.525e+00 # 2.69700000e+00  # Exponent of critical size for small colonies
q2_i =  4.138e+00 # 3.04903564e+00  # Exponent of critical size for large colonies
fit_par_i= np.array([alpha_i,ep_s1_i,ep_s2_i,q1_i,q2_i])
#fit_par_i= np.array([alpha_i,ep_s1_i])
#fit_par_i= np.array([ep_s2_i,q_i])



#Minimize penalty function to fit parameters
file_num = 4
penalty_function(folder_paths[file_num],phi_o_values[file_num],epsilon_values[file_num],fit_par_i)
#total_penalty(fit_par_i)

# Fit using all data files
#fit_result = scipy.optimize.minimize(total_penalty, fit_par_i,method='Nelder-Mead', bounds=[(0,0.2),(0.01,30),(10,200),(1.5,6),(1.5,6)], options={'maxiter': 100, 'disp': True})
#fit_file = open('fit_results.txt', 'w')  # Size distributions 
# Fit using only breakage files
#fit_result = scipy.optimize.minimize(total_penalty_bre, np.concatenate((fit_par_i[2:3],fit_par_i[4:5])),method='Nelder-Mead', bounds=[(10,200),(2,7)], options={'maxiter': 30, 'disp': True})
#fit_file = open('fit_results_breakage.txt', 'w')  # Size distributions 
# Fit using only aggregation files
#fit_result = scipy.optimize.minimize(total_penalty_agg, np.concatenate((fit_par_i[0:2],fit_par_i[3:4])),method='Nelder-Mead', bounds=[(0,0.2),(0.02,0.1),(2.0,6.0)], options={'maxiter': 50, 'disp': True})
#fit_file = open('fit_results_aggregation.txt', 'w')  # Size distributions 
# Write the fit result
#fit_file.write(str(fit_result))
#fit_file.close()




Folder: Bre_1200s_c1e6_1-5-23/Data/
penalty_function:81.560618741144


81.560618741144